# Topik Modelling Dengan Latent Semantic Indexing (LSI) atau Latent Semantic Analysis (LSA) menggunakan Scikit-Learn
Dalam pembahasan kali ini, kita akan fokus pada Latent Semantic Indexing (LSI) atau Latent Semantic Analysis (LSA) dan melakukan topik modelling menggunakan Scikit-learn.

## **Topik Modelling**
Topik Modelling ialah teknik tanpa pengawasan untuk menemukan tema dokumen yang diberikan. Ini mengekstrak kumpulan kata kunci yang terjadi bersama. Kata kunci yang muncul bersama ini mewakili sebuah topik. Misalnya, saham, pasar, ekuitas, reksa dana akan mewakili topik 'investasi saham'.

## **Latent Semantic Indexing (LSI) atau Latent Semantic Analysis (LSA)**
Latent Semantic Indexing (LSI) atau Latent Semantic Analysis (LSA)  adalah teknik dalam natural language processing , khususnya  distributional semantics , yang menganalisis hubungan antara satu set dokumen dan istilah yang dikandungnya dengan menghasilkan satu set konsep yang terkait dengan dokumen dan istilah. LSA mengasumsikan bahwa kata-kata yang memiliki makna yang dekat akan muncul dalam potongan teks yang serupa (  distributional hypothesis ). Sebuah matriks yang berisi jumlah kata per dokumen (baris mewakili kata-kata unik dan kolom mewakili setiap dokumen) dibangun dari sepotong besar teks dan teknik matematika yang disebut Singular Value Decomposition (SVD) digunakan untuk mengurangi jumlah baris dengan tetap menjaga kesamaan struktur antar kolom. Dokumen kemudian dibandingkan dengan mengambil kosinus sudut antara dua vektor (atau produk titik antara normalisasi dua vektor) yang dibentuk oleh dua kolom. Nilai yang mendekati 1 menunjukkan dokumen yang sangat mirip sedangkan nilai yang mendekati 0 menunjukkan dokumen yang sangat berbeda. Untuk melakukan LSA dapat dilakukan dengan mengikuti tahapan tahapan berikut.

## **Mengambil Dokumen**
Langkah awal untuk melakukan Topik Modelling ialah dengan mengambil dokumen tersebut dengan mengcrawling data dokumen dengan menggunakan scrapy & crochet seperti berikut.

In [1]:
!pip install scrapy
!pip install crochet

In [2]:
import scrapy
from scrapy.crawler import CrawlerRunner
import re
from crochet import setup, wait_for
setup()

class QuotesToCsv(scrapy.Spider):
    name = "MJKQuotesToCsv"
    start_urls = [
        'https://nasional.tempo.co/read/1663729/uji-kelayakan-panglima-tni-gelak-tawa-pecah-saat-yudo-margono-tunjukkan-foto-istri'
    ]
    custom_settings = {
        'ITEM_PIPELINES': {
            '__main__.ExtractFirstLine': 1
        },
        'FEEDS': {
            'news.csv': {
                'format': 'csv',
                'overwrite': True
            }
        }
    }

    def parse(self, response):
        """parse data from urls"""
        for quote in response.css('#isi > p'):
            yield {'news': quote.extract()}


class ExtractFirstLine(object):
    def process_item(self, item, spider):
        """text processing"""
        lines = dict(item)["news"].splitlines()
        first_line = self.__remove_html_tags__(lines[0])

        return {'news': first_line}

    def __remove_html_tags__(self, text):
        """remove html tags from string"""
        html_tags = re.compile('<.*?>')
        return re.sub(html_tags, '', text)

@wait_for(10)
def run_spider():
    """run spider with MJKQuotesToCsv"""
    crawler = CrawlerRunner()
    d = crawler.crawl(QuotesToCsv)
    return d

In [3]:
run_spider()

## **Meload Dokumen**
Setelah tahapan mengambil dokumen selesai, selanjutnya meload dokumen yang sudah didapatkan. Untuk dapat meload dokumen kita gunakan library os dan pandas seperti berikut.

In [4]:
import os
import pandas as pd

# Load Dataset
documents_list = []
with open( os.path.join("news.csv") ,"r") as fin:
    for line in fin.readlines():
        text = line.strip()
        documents_list.append(text)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 1003: character maps to <undefined>

## **Membuat Fitur TF-IDF**
Setelah berhasil meload dokumen langkah selanjutnya ialah mengenerate fitur TF-IDF pada dokumen. Pada proses ini juga dilakukan operasi prepocessing, yaitu case folding, stopword, dan tokenizing. Untuk melakukan proses ini dengan menggunakan RegexpTokenizer dari library nltk seperti source code berikut.

In [ ]:
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize regex tokenizer
tokenizer = RegexpTokenizer(r'\w+')

# Vectorize document using TF-IDF
tfidf = TfidfVectorizer(lowercase=True,
                        stop_words='english',
                        ngram_range = (1,1),
                        tokenizer = tokenizer.tokenize)

# Fit and Transform the documents
train_data = tfidf.fit_transform(documents_list)  
train_data

<12x236 sparse matrix of type '<class 'numpy.float64'>'
	with 412 stored elements in Compressed Sparse Row format>

## **Membuat Matrik SVD**
Matrik SVD adalah teknik dekomposisi matriks yang memfaktorkan matriks dalam produk matriks. Untuk dapat membuat matrik tersebut kita dapat menggunakan TruncatedSVD dari library sklearn seperti berikut.

In [ ]:
from sklearn.decomposition import TruncatedSVD
# Define the number of topics or components
num_components=12

# Create SVD object
lsa = TruncatedSVD(n_components=num_components, n_iter=100, random_state=42)

# Fit SVD model on data
lsa.fit_transform(train_data)

# Get Singular values and Components 
Sigma = lsa.singular_values_  
V_transpose = lsa.components_.T
V_transpose

array([[ 3.02366178e-02, -1.66382861e-02,  3.00062190e-02, ...,
         1.35817657e-01, -1.98807570e-03,  7.74902033e-03],
       [ 8.38985795e-02,  2.10631147e-01, -4.69070221e-02, ...,
         1.82148827e-02, -2.80810446e-02, -1.39858160e-01],
       [ 2.10849282e-02, -1.84815299e-02,  2.61033720e-03, ...,
        -3.35817930e-02, -4.12961820e-04, -4.39328074e-03],
       ...,
       [ 2.52767786e-02, -2.00135859e-02, -5.03940302e-02, ...,
        -3.07241369e-02,  1.88065252e-04, -2.37049787e-02],
       [ 1.58110182e-02, -2.27024106e-02,  5.80867655e-02, ...,
        -2.65079399e-02, -1.14091743e-02, -1.09058826e-02],
       [ 2.27840084e-01, -1.53073584e-01, -1.05864584e-03, ...,
        -4.25488190e-02, -4.07617741e-03,  7.17080829e-02]])

## **Ekstrak topik dan istilah**
Setelah membuar matriks SVD, Selanjutnya kita perlu mengekstrak topik dari matriks komponen SVD dengan source code seperti berikut. 

In [ ]:
# Print the topics with their terms
terms = tfidf.get_feature_names()

for index, component in enumerate(lsa.components_):
    zipped = zip(terms, component)
    top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:5]
    top_terms_list=list(dict(top_terms_key).keys())
    print("Topic "+str(index)+": ",top_terms_list)

Topic 0:  ['negara', 'dan', 'yang', 'di', 'dalam']
Topic 1:  ['elemen', 'ilmu', 'kunci', 'pengetahuan', 'adalah']
Topic 2:  ['kemudian', 'juga', 'pendekatan', 'disebut', 'berbagai']
Topic 3:  ['news', 'akan', 'sistem', 'pendekatan', 'kemudian']
Topic 4:  ['bahan', 'korea', 'sangat', 'selatan', 'taiwan']
Topic 5:  ['hal', 'baik', 'ini', 'cendekiawan', 'diskursus']
Topic 6:  ['seringkali', 'banyak', 'sektor', 'cendekiawan', 'diskursus']
Topic 7:  ['dan', 'juga', 'nies', 'antaranya', 'kelembagaan']
Topic 8:  ['dokumen', 'bahan', 'korea', 'sangat', 'selatan']
Topic 9:  ['ekonomi', 'bagian', 'begitu', 'berupaya', 'catch']
Topic 10:  ['halnya', 'indonesia', 'langsung', 'menerapkan', 'mengabsorbsi']
Topic 11:  ['amerika', 'austria', 'bagi', 'bahasa', 'belanda']
